### Cell 1: Setup and Configuration
Purpose: To centralize all settings, file paths, and analysis parameters in a single, easy-to-manage block. This makes it simple to tweak thresholds (like delta > 5%) or filenames without hunting through the code

In [1]:
# ==============================================================================
# CELL 1: SETUP AND CONFIGURATION
# ==============================================================================
import pandas as pd
import numpy as np

print("--- Cell 1: Initializing Configuration and Loading Libraries ---")

# Central dictionary for all settings, file paths, and constants.
CONFIG = {
    "input_csv_path": "00_cleaned_data.csv",
    "output_csv_path": "02_vehicle_charging_locations.csv",
    "output_map_path": "02_charging_locations_map.html",
    "high_confidence": {
        "bat_delta_threshold": 5.0,  # Min % charge increase
        "dbscan_eps_km": 0.2,        # 200m radius for clustering
        "dbscan_min_samples": 2      # Min 2 events to form a cluster
    },
    "low_confidence": {
        "bat_delta_threshold": 1.0   # Min % charge increase
    },
    "earth_radius_km": 6371.0088
}

print("Configuration loaded successfully.")

--- Cell 1: Initializing Configuration and Loading Libraries ---
Configuration loaded successfully.


### Cell 2: Data Loading and Data Health Pre-Check (Vectorized)
Purpose: To load the cleaned data and efficiently perform the crucial pre-check for data quality issues. This refactored version replaces the slow for loop with fast, vectorized groupby().transform() operations to flag stationary vehicles, which is significantly more performant.

In [2]:
# ==============================================================================
# CELL 2: DATA LOADING AND DATA HEALTH PRE-CHECK (VECTORIZED)
# ==============================================================================

print("\n--- Cell 2: Loading Data and Flagging Quality Issues ---")

# Load the data using the path from CONFIG
try:
    df = pd.read_csv(CONFIG["input_csv_path"], parse_dates=['timestamp'])
    print(f"Successfully loaded '{CONFIG['input_csv_path']}' with {len(df)} records.")
except FileNotFoundError:
    print(f"ERROR: File not found at {CONFIG['input_csv_path']}")
    df = pd.DataFrame()

if not df.empty:
    df.dropna(subset=['fixed_lat', 'fixed_long', 'fixed_bat'], inplace=True)
    df = df.sort_values(['vin', 'timestamp']).reset_index(drop=True)
    
    # --- 1. Vectorized Data Quality Check ---
    # Use transform('nunique') to count unique values for each VIN and broadcast the result.
    df['lat_nunique'] = df.groupby('vin')['fixed_lat'].transform('nunique')
    df['lon_nunique'] = df.groupby('vin')['fixed_long'].transform('nunique')
    df['bat_nunique'] = df.groupby('vin')['fixed_bat'].transform('nunique')
    
    # --- 2. Assign Issue Reasons using np.select for efficiency ---
    conditions = [
        ((df['lat_nunique'] == 1) & (df['lon_nunique'] == 1)), # Stationary GPS
        (df['bat_nunique'] == 1)                               # Stagnant Battery
    ]
    choices = ['Stationary or GPS Failure', 'Battery Signal Failure']
    df['issue_reason'] = np.select(conditions, choices, default=None)
    
    # --- 3. Create the 'Analyzable' DataFrame ---
    # This excludes all vehicles with any identified issues.
    df_analyzable = df[df['issue_reason'].isnull()].copy()
    
    # Clean up temporary helper columns
    df.drop(columns=['lat_nunique', 'lon_nunique', 'bat_nunique'], inplace=True)
    
    print(f"Found {df['issue_reason'].notna().sum()} records belonging to vehicles with data quality issues.")
    print(f"Created 'df_analyzable' with data from {df_analyzable['vin'].nunique()} healthy vehicles.")


--- Cell 2: Loading Data and Flagging Quality Issues ---
Successfully loaded '00_cleaned_data.csv' with 154116 records.
Found 6771 records belonging to vehicles with data quality issues.
Created 'df_analyzable' with data from 1045 healthy vehicles.


### Cell 3: Two-Pass Charging Location Analysis
Purpose: To perform the core analysis. This cell encapsulates the complex logic for the high-confidence (DBSCAN) and low-confidence (most frequent location) passes into clean, reusable functions. It then applies them sequentially to produce the final list of charging locations

In [3]:
# ==============================================================================
# CELL 3: TWO-PASS CHARGING LOCATION ANALYSIS
# ==============================================================================
from sklearn.cluster import DBSCAN

if 'df_analyzable' in locals() and not df_analyzable.empty:
    print("\n--- Cell 3: Two-Pass Charging Location Analysis ---")
    
    # --- 1. Calculate battery delta for the analyzable data ---
    df_analyzable['bat_delta'] = df_analyzable.groupby('vin')['fixed_bat'].diff()

    # --- 2. High-Confidence Pass (Delta > 5%) ---
    print("\nRunning High-Confidence Pass...")
    cfg_high = CONFIG["high_confidence"]
    high_conf_events = df_analyzable[df_analyzable['bat_delta'] > cfg_high["bat_delta_threshold"]].copy()
    
    def find_primary_charging_spot(group):
        if len(group) < cfg_high["dbscan_min_samples"]:
            return group[['fixed_lat', 'fixed_long']].iloc[0] # Return first event if not enough to cluster
        
        coords = np.radians(group[['fixed_lat', 'fixed_long']].values)
        epsilon = cfg_high["dbscan_eps_km"] / CONFIG["earth_radius_km"]
        db = DBSCAN(eps=epsilon, min_samples=cfg_high["dbscan_min_samples"], metric='haversine').fit(coords)
        group['cluster'] = db.labels_
        
        main_cluster = group[group['cluster'] != -1]
        if not main_cluster.empty:
            largest_id = main_cluster['cluster'].value_counts().idxmax()
            primary_spot = main_cluster[main_cluster['cluster'] == largest_id]
            return primary_spot[['fixed_lat', 'fixed_long']].mean()
        else: # Fallback if all points are noise
            return group[['fixed_lat', 'fixed_long']].iloc[0]

    high_conf_df = high_conf_events.groupby('vin').apply(find_primary_charging_spot, include_groups=False).reset_index()
    high_conf_df.rename(columns={'fixed_lat': 'charging_lat', 'fixed_long': 'charging_lon'}, inplace=True)
    high_conf_df['confidence'] = 'High'
    print(f"Found {len(high_conf_df)} high-confidence charging locations.")

    # --- 3. Low-Confidence Pass (Delta > 1%) on remaining VINs ---
    print("\nRunning Low-Confidence Pass on remaining vehicles...")
    processed_vins = set(high_conf_df['vin'])
    cfg_low = CONFIG["low_confidence"]
    
    low_conf_events = df_analyzable[
        (~df_analyzable['vin'].isin(processed_vins)) & 
        (df_analyzable['bat_delta'] > cfg_low["bat_delta_threshold"])
    ].copy()

    def find_most_frequent_location(group):
        group['lat_round'] = group['fixed_lat'].round(3)
        group['lon_round'] = group['fixed_long'].round(3)
        most_frequent = group.groupby(['lat_round', 'lon_round']).size().idxmax()
        return pd.Series({'charging_lat': most_frequent[0], 'charging_lon': most_frequent[1]})

    if not low_conf_events.empty:
        low_conf_df = low_conf_events.groupby('vin').apply(find_most_frequent_location, include_groups=False).reset_index()
        low_conf_df['confidence'] = 'Low'
        print(f"Found {len(low_conf_df)} low-confidence charging locations.")
    else:
        low_conf_df = pd.DataFrame() # Create empty df if no VINs left

    # --- 4. Combine Results ---
    found_locations_df = pd.concat([high_conf_df, low_conf_df], ignore_index=True)
    print(f"\nTotal charging locations found: {len(found_locations_df)}")


--- Cell 3: Two-Pass Charging Location Analysis ---

Running High-Confidence Pass...
Found 1002 high-confidence charging locations.

Running Low-Confidence Pass on remaining vehicles...
Found 13 low-confidence charging locations.

Total charging locations found: 1015


### Cell 4: Final Report Generation and Output
Purpose: To consolidate the analysis results into a final, comprehensive summary table. This table will include every VIN from the original dataset and assign a clear, final status (Location Found, Location Inconclusive, Data Not Found) based on the rules.

In [4]:
# ==============================================================================
# CELL 4: FINAL REPORT GENERATION AND OUTPUT
# ==============================================================================
if 'df' in locals():
    print("\n--- Cell 4: Generating Final Report for All Vehicles ---")
    
    # --- 1. Start with a unique list of all VINs and their issue reasons ---
    final_summary_df = df[['vin', 'issue_reason']].drop_duplicates().reset_index(drop=True)

    # --- 2. Merge with the charging location results ---
    if 'found_locations_df' in locals():
        final_summary_df = pd.merge(final_summary_df, found_locations_df, on='vin', how='left')
    else: # Handle case where analysis didn't run
        final_summary_df['charging_lat'] = np.nan
        final_summary_df['charging_lon'] = np.nan
        final_summary_df['confidence'] = None

    # --- 3. Assign Final Status using np.select ---
    conditions = [
        final_summary_df['issue_reason'].notna(),
        final_summary_df['charging_lat'].notna()
    ]
    choices = ['Location Inconclusive', 'Charging Location Found']
    final_summary_df['final_charging_status'] = np.select(conditions, choices, default='Data Not Found')
    
    # --- 4. Summarize and Save ---
    print("\n--- Final Status Summary for All Vehicles ---")
    print(final_summary_df['final_charging_status'].value_counts())

    # Save a clean version for output
    output_cols = ['vin', 'charging_lat', 'charging_lon', 'confidence', 'final_charging_status', 'issue_reason']
    final_summary_df[output_cols].to_csv(CONFIG["output_csv_path"], index=False)
    print(f"\nSaved final analysis to '{CONFIG['output_csv_path']}'.")
    
    display(final_summary_df[output_cols].head())


--- Cell 4: Generating Final Report for All Vehicles ---

--- Final Status Summary for All Vehicles ---
final_charging_status
Charging Location Found    1015
Location Inconclusive       190
Data Not Found               30
Name: count, dtype: int64

Saved final analysis to '02_vehicle_charging_locations.csv'.


,vin,charging_lat,charging_lon,confidence,final_charging_status,issue_reason
0,MD9EMCDL24F217385,27.237255,77.873319,High,Charging Location Found,None
1,MD9EMCDL24G217002,12.887528,80.189536,High,Charging Location Found,None
2,MD9EMCDL24G217006,28.532349,77.439695,High,Charging Location Found,None
3,MD9EMCDL24G217010,28.210756,76.838275,High,Charging Location Found,None
4,MD9EMCDL24G217011,22.703931,75.807165,High,Charging Location Found,None


### Cell 5: Visualization on Interactive Map
Purpose: To create a clear, interactive map of the results. This cell filters for the successfully identified charging locations and plots them using folium, color-coding them by confidence level (High/Low). This code was already well-structured and requires minimal changes.

In [5]:
# ==============================================================================
# CELL 5: VISUALIZATION ON INTERACTIVE MAP
# ==============================================================================
import folium

if 'final_summary_df' in locals():
    print("\n--- Cell 5: Visualizing Found Charging Locations ---")
    
    # Filter for only the vehicles with a found location
    map_data = final_summary_df[final_summary_df['final_charging_status'] == 'Charging Location Found'].copy()

    if not map_data.empty:
        # Create the base map and auto-fit to bounds
        m = folium.Map()
        sw = map_data[['charging_lat', 'charging_lon']].min().values.tolist()
        ne = map_data[['charging_lat', 'charging_lon']].max().values.tolist()
        m.fit_bounds([sw, ne])

        # Define color scheme
        def get_color(confidence):
            return 'green' if confidence == 'High' else 'orange'

        # Add points to the map
        for idx, row in map_data.iterrows():
            folium.CircleMarker(
                location=[row['charging_lat'], row['charging_lon']],
                radius=6,
                color=get_color(row['confidence']),
                fill=True, fill_color=get_color(row['confidence']), fill_opacity=0.8,
                tooltip=f"<b>VIN:</b> {row['vin']}<br><b>Confidence:</b> {row['confidence']}"
            ).add_to(m)

        # Save and display map
        m.save(CONFIG["output_map_path"])
        print(f"Map of {len(map_data)} charging locations saved to '{CONFIG['output_map_path']}'. Displaying map below.")
        display(m)
    else:
        print("No charging locations were found to plot on the map.")


--- Cell 5: Visualizing Found Charging Locations ---
Map of 1015 charging locations saved to '02_charging_locations_map.html'. Displaying map below.
